# Post Processing
1. Backup grading result.
2. Generate score report.
3. Create individual scored pdf.
4. Collect samples.
5. Email the individual scored pdf.

In [4]:
pdf_file = "../data/TestScript.pdf"

In [20]:
import os

file_name = os.path.basename(pdf_file)
file_name = os.path.splitext(file_name)[0]
base_path = "../marking_form/" + file_name
base_path_images = base_path + "/images/"
base_path_annotations = base_path+"/annotations/"
base_path_questions = base_path+"/questions"

base_path_marked_images = base_path + "/marked/images/"
base_path_marked_pdfs = base_path + "/marked/pdf/"
base_path_marked_scripts = base_path + "/marked/scripts/"
os.makedirs(base_path_marked_images, exist_ok=True)
os.makedirs(base_path_marked_pdfs, exist_ok=True)
os.makedirs(base_path_marked_scripts, exist_ok=True)


### Remove version history
Before you backup.

In [6]:
import os
for path, currentDirectory, files in os.walk(base_path_questions):
    for file in files:
        if file.startswith("control-") or file.startswith("mark-"):
            os.remove(os.path.join(path, file))

Zip the website.

In [7]:
import shutil
shutil.make_archive(base_path,"zip",base_path)

'/workspaces/ai-scoring-handwriting-assignment/marking_form/TestScript.zip'

# Generate Score Report

You must visit the ID and Name pages to verify the values before generate the marksheet.

In [8]:
import os
import json
import pandas as pd
from termcolor import colored

pageToStudentId = {}

# read base_path_annotations
with open(os.path.join(base_path_annotations, "annotations.json")) as f:
    data = json.load(f)
    # data is a dict and get the number of items
numberOfPage = len(data)

with open(os.path.join(base_path_questions, "ID", "mark.json")) as f:
    data = json.load(f)
    for i in data:
        pageToStudentId[i["id"]] = (
            i["overridedMark"] if i["overridedMark"] != "" else i["mark"]
        )


def getStudentId(page):
    # search reverse for the student ID page.
    for p in range(page, page - numberOfPage, -1):
        if str(p) in pageToStudentId:
            return pageToStudentId[str(p)]
    print(colored("{} is not in pageToStudentId!".format(page), "red"))
    return None


questionAndMarks = {}
for path, currentDirectory, files in os.walk(base_path_questions):
    for file in files:
        if file == "mark.json":
            question = path[len(base_path_questions) + 1 :]
            f = open(os.path.join(path, file))
            data = json.load(f)
            marks = {}
            for i in data:
                studentId = getStudentId(int(i["id"]))
                marks[studentId] = (
                    i["overridedMark"] if i["overridedMark"] != "" else i["mark"]
                )
            questionAndMarks[question] = marks
            f.close()
marksDf = pd.DataFrame(questionAndMarks)
marksDf = marksDf[
    ["ID", "NAME", "CLASS"]
    + [
        col
        for col in sorted(marksDf.columns)
        if col != "ID" and col != "NAME" and col != "CLASS"
    ]
]

marksDf["Marks"] = (
    marksDf.loc[:, ~marksDf.columns.isin(["ID", "NAME", "CLASS"])]
    .apply(pd.to_numeric)
    .sum(axis=1)
)
print(marksDf)

                  ID           NAME CLASS Q1 Q36 Q37 Q38  Marks
200275822  200275822    LEUNGTszHin        0   0   0   0      0
200241763  200241763     TAMHinChun        0   0   0   0      0
200238088  200238088      LoChunHei     A  1   0   0   0      1
200186026  200186026   YUEN WingChu   E&L  0   0   0   0      0
200091406  200091406    CHEUNGWaKin   E&L  0   0   0   0      0
200035120  200035120   TSANGYukKuer     A  0   1   0   0      1
200177460  200177460    YIMWaiChung   EaL  0   0   0   0      0
200081960  200081960    HUNGKamFung   E&L  0   0   0   0      0
200365446  200365446    CHEUNGHoShi     A  0   0   0   0      0
200429237  200429237      LUOMianan     A  0   0   0   0      0
200144594  200144594    CHANPakHong        0   0   0   0      0
200276566  200276566     NGChunWing        0   0   0   0      0
200167780  200167780        LOPakHo   ESL  0   0   0   0      0
200133094  200133094       SOKaChun     A  0   0   0   0      0
200532530  200532530  KWAN Wing Sze     

# Create Scored Scripts

Copy raw images to marked folder

In [9]:
import shutil
import os

if os.path.exists(base_path_marked_images):
    shutil.rmtree(base_path_marked_images)
shutil.copytree(base_path_images, base_path_marked_images)

'../marking_form/TestScript/marked/images/'

In [12]:
import json
annotations_path = base_path_annotations + "annotations.json"
with open(annotations_path, "r") as f: 
    annotations = json.load(f)          

#flatten annotations to list 
annotations_list = []
for page in annotations:
    for annotation in annotations[page]:
        annotation["page"] = int(page)
        # x to left, y to top
        annotation["left"] = annotation["x"]
        annotation["top"] = annotation["y"]
        annotation.pop("x")
        annotation.pop("y")
        annotations_list.append(annotation) 
annotations_list

# convert annotations_list to dict with key with label
annotations_dict = {}
for annotation in annotations_list:
    annotations_dict[annotation["label"]] = annotation
annotations_dict


{'NAME': {'width': 322,
  'height': 77,
  'label': 'NAME',
  'page': 0,
  'left': 238,
  'top': 173},
 'ID': {'width': 471,
  'height': 83,
  'label': 'ID',
  'page': 0,
  'left': 175,
  'top': 259},
 'CLASS': {'width': 168,
  'height': 91,
  'label': 'CLASS',
  'page': 0,
  'left': 835,
  'top': 248},
 'Q1': {'width': 322,
  'height': 111,
  'label': 'Q1',
  'page': 0,
  'left': 164,
  'top': 344},
 'Q36': {'width': 347,
  'height': 138,
  'label': 'Q36',
  'page': 1,
  'left': 184,
  'top': 482},
 'Q37': {'width': 410,
  'height': 150,
  'label': 'Q37',
  'page': 1,
  'left': 665,
  'top': 460},
 'Q38': {'width': 384,
  'height': 143,
  'label': 'Q38',
  'page': 1,
  'left': 1166,
  'top': 473}}

In [10]:
studentIdToPage={}
with open(os.path.join(base_path_questions, "ID", "mark.json")) as f:
    data = json.load(f)
    for i in data:
        studentId = i["overridedMark"] if i["overridedMark"] != "" else i["mark"]
        studentIdToPage[studentId] = int(i["id"])
studentIdToPage


{'200012886': 0,
 '200238088': 10,
 '200021312': 100,
 '200017034': 102,
 '200237504': 104,
 '200232139': 106,
 '200005883': 108,
 '200532530': 110,
 '200144594': 112,
 '200048545': 12,
 '200072256': 14,
 '200266235': 16,
 '200119573': 18,
 '200365446': 2,
 '200096278': 20,
 '200084507': 22,
 '200169933': 24,
 '200236532': 26,
 '200149381': 28,
 '200143500': 30,
 '200188622': 32,
 '200185933': 34,
 '200167780': 36,
 '200189249': 38,
 '200147191': 4,
 '200337380': 40,
 '200186026': 42,
 '200429237': 44,
 '200159097': 46,
 '200529119': 48,
 '200099220': 50,
 '200169146': 52,
 '200169171': 54,
 '200133094': 56,
 '200276566': 58,
 '200035132': 6,
 '200091406': 60,
 '200006867': 62,
 '200035120': 64,
 '200275822': 66,
 '200172741': 68,
 '200082391': 70,
 '200086758': 72,
 '200241763': 74,
 '200177527': 76,
 '200023631': 78,
 '200177460': 8,
 '200270390': 80,
 '200228716': 82,
 '200084821': 84,
 '200187164': 86,
 '200081960': 88,
 '200148151': 90,
 '200277378': 92,
 '200358399': 94,
 '200239

In [13]:
import cv2
from IPython.display import display
from ipywidgets import IntProgress


# Covert marksDf to dict
marksDf_list = marksDf.to_dict(orient="records")

f = IntProgress(min=0, max=len(marksDf_list))  # instantiate the bar
display(f)  # display the bar

for student in marksDf_list:
    first_page = studentIdToPage[student["ID"]]
    for annotation in annotations_dict:
        value = student[annotation]
        x = annotations_dict[annotation]["left"]
        y = annotations_dict[annotation]["top"]
        page = first_page + annotations_dict[annotation]["page"]
      
        image_path = base_path_marked_images + str(page) + ".jpg"
        # print(value, x, y, imagePath)
        img = cv2.imread(image_path)
        cv2.putText(img, str(value), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imwrite(image_path, img)
    f.value += 1      



IntProgress(value=0, max=57)

In [14]:
from PIL import Image

for student in marksDf_list:
    studentId = student["ID"]
    first_page = studentIdToPage[student["ID"]]
    last_page = first_page + numberOfPage - 1
    print(studentId, first_page, last_page)
    pdf_path = base_path_marked_pdfs + studentId + ".pdf"

    images = list(map(Image.open, [base_path_marked_images + str(i) + ".jpg" for i in range(first_page, last_page + 1)]))
    images[0].save(pdf_path, save_all=True, append_images=images[1:]) 
        

200275822 66 67
200241763 74 75
200238088 10 11
200186026 42 43
200091406 60 61
200035120 64 65
200177460 8 9
200081960 88 89
200365446 2 3
200429237 44 45
200144594 112 113
200276566 58 59
200167780 36 37
200133094 56 57
200532530 110 111
200048545 12 13
200072256 14 15
200096278 20 21
200188622 32 33
200529119 48 49
200159097 46 47
200169146 52 53
200187164 86 87
200017034 102 103
200086758 72 73
200012886 0 1
200021312 100 101
200237504 104 105
200232139 106 107
200005883 108 109
200266235 16 17
200119573 18 19
200084507 22 23
200169933 24 25
200236532 26 27
200149381 28 29
200143500 30 31
200185933 34 35
200189249 38 39
200337380 40 41
200147191 4 5
200099220 50 51
200169171 54 55
200006867 62 63
200172741 68 69
200035132 6 7
200082391 70 71
200177527 76 77
200023631 78 79
200270390 80 81
200228716 82 83
200084821 84 85
200148151 90 91
200277378 92 93
200358399 94 95
200239558 96 97
200095386 98 99


# Generate Script Sample

5 set Samples:
1. Combined scripts
2. 3 Good, 3 Average, and 3 Weak.
3. 5 Good, 5 Average, and 5 Weak.
4. 3 Good, 3 Average, and 3 Weak above the passing mark.
5. 5 Good, 5 Average, and 5 Weak above the passing mark. 

In [15]:
passingMark = 0

In [23]:
from PyPDF4 import PdfFileMerger

writer = PdfFileMerger(strict=True)

# merge all pdfs in base_path_marked_pdfs
for path, currentDirectory, files in os.walk(base_path_marked_pdfs):
    for file in files:
        if file.endswith(".pdf"):
            pdf_path = os.path.join(path, file)
            writer.append(pdf_path)
writer.write(base_path_marked_scripts + "all.pdf")           

In [22]:
from PyPDF4 import PdfFileMerger, PdfFileReader

sampling = marksDf.sort_values(by=["Marks"], ascending=False)["Marks"]

from_directory = os.path.join(os.getcwd(), "..", "templates", "pdf")

goodPage = PdfFileReader(from_directory + "/Good.pdf")
averagePage = PdfFileReader(from_directory + "/Average.pdf")
weakPage = PdfFileReader(from_directory + "/Weak.pdf")


def get_scripts_psf(df):
    return list(map(lambda rowNumber: base_path_marked_pdfs + rowNumber + ".pdf", df.index))


def take_sample(n, sampling, suffix=""):
    if len(sampling) < 3 * n:
        n = int(len(sampling) / 3)
    good = sampling.head(n)
    weak = sampling.tail(n)
    median = int(len(sampling) / 2)
    take = int(n / 2)
    average = sampling.iloc[median - take : median + take]

    merger = PdfFileMerger()
    merger.append(goodPage)
    for pdf in get_scripts_psf(good):
        merger.append(PdfFileReader(pdf))
    merger.append(averagePage)
    for pdf in get_scripts_psf(average):
        merger.append(PdfFileReader(pdf))
    merger.append(weakPage)
    for pdf in get_scripts_psf(weak):
        merger.append(PdfFileReader(pdf))
    fileName = base_path_marked_scripts + "sampleOf" + str(n) + suffix + ".pdf"
    merger.write(open(fileName, "wb"))
    print("Output successfully written to" + fileName)
    merger.close()


take_sample(3, sampling)
take_sample(5, sampling)

sampling = sampling.where(lambda x: x > passingMark)
take_sample(3, sampling, "_only_pass")
take_sample(5, sampling, "_only_pass")


Output successfully written to../marking_form/TestScript/marked/scripts/sampleOf3.pdf
Output successfully written to../marking_form/TestScript/marked/scripts/sampleOf5.pdf
Output successfully written to../marking_form/TestScript/marked/scripts/sampleOf3_only_pass.pdf
Output successfully written to../marking_form/TestScript/marked/scripts/sampleOf5_only_pass.pdf


In [25]:
# save marksDf to excel
marksDf.to_excel(base_path_marked_scripts + "details_score_report.xlsx", index=False)
# save marksDf to excel but only show ID, NAME, CLASS, Marks
marksDf[["ID", "NAME", "CLASS", "Marks"]].to_excel(base_path_marked_scripts + "score_report.xlsx", index=False)

In [34]:
from IPython.display import FileLink 

# zip base_path_marked_scripts folder
script_zip = base_path_marked_scripts + "../scripts"
shutil.make_archive(script_zip, "zip", base_path_marked_scripts)
FileLink(script_zip + ".zip")

/workspaces/ai-scoring-handwriting-assignment/marking_form/TestScript/marked/scripts.zip